# Final Project

#### Install missing library

In [3]:
#!pip install tabulate

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score, accuracy_score, f1_score
from tabulate import tabulate

#### Import data

In [5]:
Monday_df = pd.read_csv('archive_2/Monday_WorkingHours.csv')
Monday_df = pd.DataFrame(Monday_df)
print(Monday_df.shape) 
Monday_df.columns = Monday_df.columns.str.strip() #Remove extra spaces from column names

Tuesday_BruteForce = pd.read_csv('archive_2/Tuesday_BruteForce.csv')
Tuesday_BruteForce = pd.DataFrame(Tuesday_BruteForce)
print(Tuesday_BruteForce.shape)
Tuesday_BruteForce.columns = Tuesday_BruteForce.columns.str.strip()

Wednesday_DOS = pd.read_csv('archive_2/Wednesday_DOS_DDOS.csv')
Wednesday_DOS = pd.DataFrame(Wednesday_DOS)
print(Wednesday_DOS.shape)
Wednesday_DOS.columns = Wednesday_DOS.columns.str.strip() 

Thursday_WebAttack = pd.read_csv('archive_2/Thursday_Morning_WebAttacks.csv')
Thursday_WebAttack = pd.DataFrame(Thursday_WebAttack)
print(Thursday_WebAttack.shape)
Thursday_WebAttack.columns = Thursday_WebAttack.columns.str.strip()

Thursday_Infiltration = pd.read_csv('archive_2/Thursday_Afternoon_Infiltration.csv')
Thursday_Infiltration = pd.DataFrame(Thursday_Infiltration)
print(Thursday_Infiltration.shape)
Thursday_Infiltration.columns = Thursday_Infiltration.columns.str.strip()

Friday_Botnet = pd.read_csv('archive_2/Friday_Morning_Botnet.csv')
Friday_Botnet = pd.DataFrame(Friday_Botnet)
print(Friday_Botnet.shape)
Friday_Botnet.columns = Friday_Botnet.columns.str.strip()

Friday_Portscan = pd.read_csv('archive_2/Friday_Afternoon_PortScan.csv')
Friday_Portscan = pd.DataFrame(Friday_Portscan)
print(Friday_Portscan.shape)
Friday_Portscan.columns = Friday_Portscan.columns.str.strip()

Friday_DDOS = pd.read_csv('archive_2/Friday_Afternoon_DDos.csv')
Friday_DDOS = pd.DataFrame(Friday_DDOS)
print(Friday_DDOS.shape)
Friday_DDOS.columns = Friday_DDOS.columns.str.strip()

(529918, 79)
(445909, 79)
(692703, 79)
(170366, 79)
(288602, 79)
(191033, 79)
(286467, 79)
(225745, 79)


#### Explore columns to be removed

In [6]:
def analyzeFeatures(df):
    results = {}
    # Duplicate columns
    checked = []
    duplicate_cols = []
    for col1 in df.columns:
        for col2 in df.columns:
            if col1 != col2 and col2 not in checked:
                if df[col1].equals(df[col2]):
                    duplicate_cols.append((col1,col2))
                    checked.append(col2)
    results['duplicates'] = duplicate_cols

    #Not unique columns
    constant_cols = [col for col in df.columns if df[col].nunique() == 1]
    results['constant_cols'] = constant_cols

    #Missing value columns
    missing = df.isna().mean().sort_values(ascending=False)
    results['missing'] = missing[missing > 0].to_dict()

    return results

Mon_features = analyzeFeatures(Monday_df)
BruteForce_features = analyzeFeatures(Tuesday_BruteForce)
DOS_features = analyzeFeatures(Wednesday_DOS)
WebAttack_features = analyzeFeatures(Thursday_WebAttack)
Infiltration_features = analyzeFeatures(Thursday_Infiltration)
Botnet_features = analyzeFeatures(Friday_Botnet)
Portscan_features = analyzeFeatures(Friday_Portscan)
DDOS_features = analyzeFeatures(Friday_DDOS)

dataset_features = [Mon_features, BruteForce_features, DOS_features, WebAttack_features, Infiltration_features, Botnet_features, Portscan_features, DDOS_features]

def feature_summary(dataset_features):
    duplicate_lists = [set(map(tuple, d['duplicates'])) for d in dataset_features]
    constant_lists = [set(d['constant_cols']) for d in dataset_features]
    missing_lists = [set(d['missing'].keys()) for d in dataset_features]

    common_duplicates = set.intersection(*duplicate_lists) if duplicate_lists else set()
    common_constants = set.intersection(*constant_lists) if constant_lists else set()
    common_missing = set.intersection(*missing_lists) if missing_lists else set()

    return {
        'common_duplicate': sorted(list(common_duplicates)),
        'common_constant': sorted(list(common_constants)),
        'common_missing': sorted(list(common_missing))
    }

summary = feature_summary(dataset_features)
print(summary)

{'common_duplicate': [('Bwd PSH Flags', 'Bwd Avg Bulk Rate'), ('Bwd PSH Flags', 'Bwd Avg Bytes/Bulk'), ('Bwd PSH Flags', 'Bwd Avg Packets/Bulk'), ('Bwd PSH Flags', 'Bwd URG Flags'), ('Bwd PSH Flags', 'Fwd Avg Bulk Rate'), ('Bwd PSH Flags', 'Fwd Avg Bytes/Bulk'), ('Bwd PSH Flags', 'Fwd Avg Packets/Bulk'), ('Fwd Header Length', 'Fwd Header Length.1'), ('Fwd Header Length.1', 'Fwd Header Length'), ('Fwd PSH Flags', 'SYN Flag Count'), ('SYN Flag Count', 'Fwd PSH Flags'), ('Subflow Bwd Packets', 'Total Backward Packets'), ('Subflow Fwd Packets', 'Total Fwd Packets'), ('Total Backward Packets', 'Subflow Bwd Packets'), ('Total Fwd Packets', 'Subflow Fwd Packets')], 'common_constant': ['Bwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd PSH Flags', 'Bwd URG Flags', 'Fwd Avg Bulk Rate', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk'], 'common_missing': ['Flow Bytes/s']}


#### Drop columns and create y dataframe

In [7]:
def copyLabelDropColumns(df):
    yDF = df[['Label']].copy()
    newDF = df.drop(['Destination Port', 'Label', 'Bwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd PSH Flags', 'Bwd URG Flags', 'Fwd Avg Bulk Rate', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Header Length.1', 'Fwd PSH Flags', 'Subflow Bwd Packets','Subflow Fwd Packets'], axis=1)
    return yDF, newDF

In [8]:
#Monday
Mon_df_y, Mon_drop_df = copyLabelDropColumns(Monday_df)
#print(Mon_df_y[:5])
print(Mon_drop_df.shape)

#Tuesday Brute Force
BruteForce_df_y, BruteForce_drop_df = copyLabelDropColumns(Tuesday_BruteForce)
#print(BruteForce_df_y[:5])
print(BruteForce_drop_df.shape)

#Wednesday DOS
DOS_df_y, DOS_drop_df = copyLabelDropColumns(Wednesday_DOS)
#print(DOS_df_y[:5])
print(DOS_drop_df.shape)

#Thursday Web Attack
WebAttack_df_y, WebAttack_drop_df = copyLabelDropColumns(Thursday_WebAttack)
#print(WebAttack_df_y[:5])
print(WebAttack_drop_df.shape)

#Thursday Infiltration
Infiltration_df_y, Infiltration_drop_df = copyLabelDropColumns(Thursday_Infiltration)
#print(Infiltration_df_y[:5])
print(Infiltration_drop_df.shape)

#Friday Botnet
Botnet_df_y, Botnet_drop_df = copyLabelDropColumns(Friday_Botnet)
#print(Botnet_df_y[:5])
print(Botnet_drop_df.shape)

#Friday Portscan
Portscan_df_y, Portscan_drop_df = copyLabelDropColumns(Friday_Portscan)
#print(Portscan_df_y[:5])
print(Portscan_drop_df.shape)

#Friday DDOS
DDOS_df_y, DDOS_drop_df = copyLabelDropColumns(Friday_DDOS)
#print(DDOS_df_y[:5])
print(DDOS_drop_df.shape)

(529918, 65)
(445909, 65)
(692703, 65)
(170366, 65)
(288602, 65)
(191033, 65)
(286467, 65)
(225745, 65)


#### What columns are left

In [9]:
print(DDOS_drop_df.columns)

Index(['Flow Duration', 'Total Fwd Packets', 'Total Backward Packets',
       'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd URG Flags',
       'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s',
       'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length',
       'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance',
       'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count',
       'ACK Flag Count', 'URG Flag Count', 

#### Encode Labels

In [10]:
def encodeLabels(df):
    df['Label_encoding'] = df['Label'].apply(lambda x: 0 if x == 'BENIGN' else 1)
    df = df.drop(['Label'], axis=1)
    return df

Mon_df_y = encodeLabels(Mon_df_y)
#Make into series
Mon_df_y = Mon_df_y['Label_encoding'] 
#print(Mon_df_y[:5])

BruteForce_df_y = encodeLabels(BruteForce_df_y)
#Make into series
BruteForce_df_y = BruteForce_df_y['Label_encoding'] 
#print(BruteForce_df_y[:5])

DOS_df_y = encodeLabels(DOS_df_y)
#Make into series
DOS_df_y = DOS_df_y['Label_encoding'] 
#print(DOS_df_y[:5])

WebAttack_df_y = encodeLabels(WebAttack_df_y)
#Make into series
WebAttack_df_y = WebAttack_df_y['Label_encoding']
#print(WebAttack_df_y[:5])

Infiltration_df_y = encodeLabels(Infiltration_df_y)
#Make into series
Infiltration_df_y = Infiltration_df_y['Label_encoding']
#print(Infiltration_df_y[:5])

Botnet_df_y = encodeLabels(Botnet_df_y)
#Make into series
Botnet_df_y = Botnet_df_y['Label_encoding']
#print(Botnet_df_y[:5])

Portscan_df_y = encodeLabels(Portscan_df_y)
#Make into series
Portscan_df_y = Portscan_df_y['Label_encoding']
#print(Portscan_df_y[:5])

DDOS_df_y = encodeLabels(DDOS_df_y)
#Make into series
DDOS_df_y = DDOS_df_y['Label_encoding']
#print(DDOS_df_y[:5])


#### Check that correct labels are maintained

In [11]:
#print('Monday: ', '\n', Mon_df_y['Label_encoding'].value_counts())
#print('BruteForce:', '\n',  BruteForce_df_y['Label_encoding'].value_counts())
#print('DOS: ', '\n', DOS_df_y['Label_encoding'].value_counts())
#print('WebAttack: ', '\n', WebAttack_df_y['Label_encoding'].value_counts())
#print('Infiltration: ', '\n', Infiltration_df_y['Label_encoding'].value_counts())
#print('Botnet: ', '\n', Botnet_df_y['Label_encoding'].value_counts())
#print('Portscan: ', '\n', Portscan_df_y['Label_encoding'].value_counts())
#print('DDOS: ', '\n', DDOS_df_y['Label_encoding'].value_counts())

#### Train test split all data

In [12]:
#Monday train, test split
X_Mon_train, X_Mon_test, Y_Mon_train, Y_Mon_test = train_test_split(
    Mon_drop_df, Mon_df_y, 
    test_size=.2, 
    stratify=Mon_df_y, 
    random_state=28)

In [13]:
#BruteForce train, test split
X_BruteForce_train, X_BruteForce_test, Y_BruteForce_train, Y_BruteForce_test = train_test_split(
    BruteForce_drop_df, BruteForce_df_y, 
    test_size=.2, 
    stratify=BruteForce_df_y, 
    random_state=28)

In [14]:
#DOS train, test split
X_DOS_train, X_DOS_test, Y_DOS_train, Y_DOS_test = train_test_split(
    DOS_drop_df, DOS_df_y, 
    test_size=.2, 
    stratify=DOS_df_y, 
    random_state=28)

In [15]:
#WebAttack train, test split
X_WebAttack_train, X_WebAttack_test, Y_WebAttack_train, Y_WebAttack_test = train_test_split(
    WebAttack_drop_df, WebAttack_df_y, 
    test_size=.2, 
    stratify=WebAttack_df_y, 
    random_state=28)

In [16]:
#Infiltration train, test split
X_Infiltration_train, X_Infiltration_test, Y_Infiltration_train, Y_Infiltration_test = train_test_split(
    Infiltration_drop_df, Infiltration_df_y, 
    test_size=.2, 
    stratify=Infiltration_df_y, 
    random_state=28)

In [17]:
#Botnet train, test split
X_Botnet_train, X_Botnet_test, Y_Botnet_train, Y_Botnet_test = train_test_split(
    Botnet_drop_df, Botnet_df_y, 
    test_size=.2, 
    stratify=Botnet_df_y, 
    random_state=28)

In [18]:
#Portscan train, test split
X_Portscan_train, X_Portscan_test, Y_Portscan_train, Y_Portscan_test = train_test_split(
    Portscan_drop_df, Portscan_df_y, 
    test_size=.2, 
    stratify=Portscan_df_y, 
    random_state=28)


In [19]:
#Portscan train, test split
X_DDOS_train, X_DDOS_test, Y_DDOS_train, Y_DDOS_test = train_test_split(
    DDOS_drop_df, DDOS_df_y, 
    test_size=.2, 
    stratify=DDOS_df_y, 
    random_state=28)


#### Add Monday to all attack days

In [20]:
def addMonday(x_train, x_test, y_train, y_test):
    concat_x_train = pd.concat([X_Mon_train, x_train], ignore_index=True)
    concat_x_test = pd.concat([X_Mon_test, x_test], ignore_index=True)
    concat_y_train = pd.concat([Y_Mon_train, y_train], ignore_index=True)
    concat_y_test = pd.concat([Y_Mon_test, y_test], ignore_index=True)
    return concat_x_train, concat_x_test, concat_y_train, concat_y_test

In [21]:
#Concat BruteForce
BruteForceMon_x_train, BruteForceMon_x_test, BruteForceMon_y_train, BruteForceMon_y_test = addMonday(X_BruteForce_train, X_BruteForce_test, Y_BruteForce_train, Y_BruteForce_test)

#Concat DOS
DOSMon_x_train, DOSMon_x_test, DOSMon_y_train, DOSMon_y_test = addMonday(X_DOS_train, X_DOS_test, Y_DOS_train, Y_DOS_test)

#Concat WebAttack
WebAttackMon_x_train, WebAttackMon_x_test, WebAttackMon_y_train, WebAttackMon_y_test = addMonday(X_WebAttack_train, X_WebAttack_test, Y_WebAttack_train, Y_WebAttack_test)

#Concat Infiltration
InfiltrationMon_x_train, InfiltrationMon_x_test, InfiltrationMon_y_train, InfiltrationMon_y_test = addMonday(X_Infiltration_train, X_Infiltration_test, Y_Infiltration_train, Y_Infiltration_test)

#Concat Botnet
BotnetMon_x_train, BotnetMon_x_test, BotnetMon_y_train, BotnetMon_y_test = addMonday(X_Botnet_train, X_Botnet_test, Y_Botnet_train, Y_Botnet_test)

#Concat Portscan
PortscanMon_x_train, PortscanMon_x_test, PortscanMon_y_train, PortscanMon_y_test = addMonday(X_Portscan_train, X_Portscan_test, Y_Portscan_train, Y_Portscan_test)

#Concat DDOS
DDOSMon_x_train, DDOSMon_x_test, DDOSMon_y_train, DDOSMon_y_test = addMonday(X_DDOS_train, X_DDOS_test, Y_DDOS_train, Y_DDOS_test)

#### Scale features

In [22]:
def scaleFeatures(train, test):
    # Replace infinities with NaN
    train = train.replace([np.inf, -np.inf], np.nan)
    test  = test.replace([np.inf, -np.inf], np.nan)

    # Handle NaNs by filling with median of column
    train_medians = train.median()
    train = train.fillna(train_medians)
    test  = test.fillna(train_medians)

    #Declare scaler, fit on training
    scaler = MinMaxScaler().fit(train)

    #Transform train, test
    train_scaled = pd.DataFrame(scaler.transform(train), columns=train.columns)
    test_scaled  = pd.DataFrame(scaler.transform(test), columns=test.columns)

    return train_scaled, test_scaled

#Scale BruteForce
BruteForceMon_x_train, BruteForceMon_x_test = scaleFeatures(BruteForceMon_x_train, BruteForceMon_x_test)
#Scale DOS
DOSMon_x_train, DOSMon_x_test = scaleFeatures(DOSMon_x_train, DOSMon_x_test)
#Scale WebAttack
WebAttackMon_x_train, WebAttackMon_x_test = scaleFeatures(WebAttackMon_x_train, WebAttackMon_x_test)
#Scale Infiltration
InfiltrationMon_x_train, InfiltrationMon_x_test = scaleFeatures(InfiltrationMon_x_train, InfiltrationMon_x_test)
#Scale Botnet
BotnetMon_x_train, BotnetMon_x_test = scaleFeatures(BotnetMon_x_train, BotnetMon_x_test)
#Scale Portscan
PortscanMon_x_train, PortscanMon_x_test = scaleFeatures(PortscanMon_x_train, PortscanMon_x_test)
#Scale DDOS
DDOSMon_x_train, DDOSMon_x_test = scaleFeatures(DDOSMon_x_train, DDOSMon_x_test)

#### The Models

In [23]:
#Logistic Regression
logistic = LogisticRegression(max_iter=2000, class_weight='balanced')

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
scoring = {'AUC': 'roc_auc', 'Accuracy': 'accuracy', 'Average Precision': 'average_precision', 'F1': 'f1'}

param_grid_logistic = [
   {
        'penalty': ['l2', 'l1'], 
        'C': [0.01, 0.1, 1, 10],
        'solver': ['saga']
    }
]

# Setting refit='AUC', refits an estimator on the whole dataset with the
# parameter setting that has the best cross-validated AUC score.
# That estimator is made available at ``gs.best_estimator_`` along with
# parameters like ``gs.best_score_``, ``gs.best_params_`` and
# ``gs.best_index_``
grid_search_logistic = GridSearchCV(
    logistic, 
    param_grid=param_grid_logistic,
    refit='Average Precision', #refits estimator on the whole dataset with parameter setting
    cv=cv,
    n_jobs=1,
    scoring=scoring
)


In [24]:
#KNN
knn = KNeighborsClassifier()
param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto'],
    'leaf_size': [30, 40]
}
grid_search_knn = GridSearchCV(
    knn,
    param_grid=param_grid_knn,
    refit='Average Precision',
    cv=cv,
    n_jobs=1,
    scoring=scoring
)

In [25]:
#Random Forest Classifier
random = RandomForestClassifier(random_state=42, class_weight='balanced')

param_grid_rf = {
    'n_estimators': [200, 500],       
    'max_depth': [None, 15, 30],      
    'min_samples_split': [2, 5],      
    'min_samples_leaf': [1, 2],       
    'max_features': ['sqrt'] 
}

grid_search_rf = GridSearchCV(
    random,
    param_grid=param_grid_rf,
    refit='Average Precision',
    cv=cv,
    n_jobs=1,
    scoring=scoring   
)

In [26]:
#MLP Classifier
mlpclassifier = MLPClassifier(max_iter=2000)

param_grid_mlp = {
   'hidden_layer_sizes': [(64,), (128,), (128, 64)],
    'activation': ['relu'],     
    'solver': ['adam'],         
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['adaptive']
}

grid_search_mlp = GridSearchCV(
   mlpclassifier,
   param_grid=param_grid_mlp,
   refit='Average Precision',
   cv=cv,
   n_jobs=1,
   scoring=scoring
)

#### Brute Force Analysis

In [27]:
#Hyperparameter training subset split
"""
BruteForce_x_train_subset, _, BruteForce_y_train_subset, _ = train_test_split(
    BruteForceMon_x_train, BruteForceMon_y_train,
    train_size=0.05,            
    stratify=BruteForceMon_y_train,          
    random_state=42
)

#Logistic Regression
grid_search_logistic.fit(BruteForce_x_train_subset, BruteForce_y_train_subset)
BF_logistic_bestestimator = grid_search_logistic.best_estimator_
BF_logistic_bestparams = grid_search_logistic.best_params_
BF_logistic_bestscore = grid_search_logistic.best_score_
print('Brute Force')
print('Logistic Regression')
print("Best parameters:", BF_logistic_bestparams)
print("Best cross-val score: {:.2f}%".format(BF_logistic_bestscore * 100))
print('\n')

#KNN
grid_search_knn.fit(BruteForce_x_train_subset, BruteForce_y_train_subset)
BF_knn_bestestimator = grid_search_knn.best_estimator_
BF_knn_bestparams = grid_search_knn.best_params_
BF_knn_bestscore = grid_search_knn.best_score_
print('KNN')
print("Best parameters:", BF_knn_bestparams)
print("Best score: {:.2f}%".format(BF_knn_bestscore * 100))
print('\n')

#Random Forest Classifier
grid_search_rf.fit(BruteForce_x_train_subset, BruteForce_y_train_subset)
BF_rf_bestestimator = grid_search_rf.best_estimator_
BF_rf_bestparams = grid_search_rf.best_params_
BF_rf_bestscore = grid_search_rf.best_score_
print('Random Forest')
print("Best parameters:", BF_rf_bestparams)
print("Best score : {:.2f}%".format(BF_rf_bestscore * 100))
print('\n')

#MLP Classifier
grid_search_mlp.fit(BruteForce_x_train_subset, BruteForce_y_train_subset)
BF_mlp_bestestimator = grid_search_mlp.best_estimator_
BF_mlp_bestparams = grid_search_mlp.best_params_
BF_mlp_bestscore = grid_search_mlp.best_score_
print('Multi-Layer Perceptron')
print("Best parameters:", BF_mlp_bestparams)
print("Best score: {:.2f}%".format(BF_mlp_bestscore * 100))
"""

'\nBruteForce_x_train_subset, _, BruteForce_y_train_subset, _ = train_test_split(\n    BruteForceMon_x_train, BruteForceMon_y_train,\n    train_size=0.05,            \n    stratify=BruteForceMon_y_train,          \n    random_state=42\n)\n\n#Logistic Regression\ngrid_search_logistic.fit(BruteForce_x_train_subset, BruteForce_y_train_subset)\nBF_logistic_bestestimator = grid_search_logistic.best_estimator_\nBF_logistic_bestparams = grid_search_logistic.best_params_\nBF_logistic_bestscore = grid_search_logistic.best_score_\nprint(\'Brute Force\')\nprint(\'Logistic Regression\')\nprint("Best parameters:", BF_logistic_bestparams)\nprint("Best cross-val score: {:.2f}%".format(BF_logistic_bestscore * 100))\nprint(\'\n\')\n\n#KNN\ngrid_search_knn.fit(BruteForce_x_train_subset, BruteForce_y_train_subset)\nBF_knn_bestestimator = grid_search_knn.best_estimator_\nBF_knn_bestparams = grid_search_knn.best_params_\nBF_knn_bestscore = grid_search_knn.best_score_\nprint(\'KNN\')\nprint("Best parameters

#### DOS Analysis

In [28]:
#Hyperparameter training subset split
"""
DOS_x_train_subset, _, DOS_y_train_subset, _ = train_test_split(
    DOSMon_x_train, DOSMon_y_train,
    train_size=0.05,            
    stratify=DOSMon_y_train,          
    random_state=42
)

#Logistic Regression
grid_search_logistic.fit(DOS_x_train_subset, DOS_y_train_subset)
DOS_logistic_bestestimator = grid_search_logistic.best_estimator_
DOS_logistic_bestparams = grid_search_logistic.best_params_
DOS_logistic_bestscore = grid_search_logistic.best_score_
print('DOS')
print('Logistic Regression')
print("Best parameters:", DOS_logistic_bestparams)
print("Best cross-val score: {:.2f}%".format(DOS_logistic_bestscore * 100))
print('\n')

#KNN
grid_search_knn.fit(DOS_x_train_subset, DOS_y_train_subset)
DOS_knn_bestestimator = grid_search_knn.best_estimator_
DOS_knn_bestparams = grid_search_knn.best_params_
DOS_knn_bestscore = grid_search_knn.best_score_
print('KNN')
print("Best parameters:", DOS_knn_bestparams)
print("Best score: {:.2f}%".format(DOS_knn_bestscore * 100))
print('\n')

#Random Forest Classifier
grid_search_rf.fit(DOS_x_train_subset, DOS_y_train_subset)
DOS_rf_bestestimator = grid_search_rf.best_estimator_
DOS_rf_bestparams = grid_search_rf.best_params_
DOS_rf_bestscore = grid_search_rf.best_score_
print('Random Forest')
print("Best parameters:", DOS_rf_bestparams)
print("Best score : {:.2f}%".format(DOS_rf_bestscore * 100))
print('\n')

#MLP Classifier
grid_search_mlp.fit(DOS_x_train_subset, DOS_y_train_subset)
DOS_mlp_bestestimator = grid_search_mlp.best_estimator_
DOS_mlp_bestparams = grid_search_mlp.best_params_
DOS_mlp_bestscore = grid_search_mlp.best_score_
print('Multi-Layer Perceptron')
print("Best parameters:", DOS_mlp_bestparams)
print("Best score: {:.2f}%".format(DOS_mlp_bestscore * 100))
"""

'\nDOS_x_train_subset, _, DOS_y_train_subset, _ = train_test_split(\n    DOSMon_x_train, DOSMon_y_train,\n    train_size=0.05,            \n    stratify=DOSMon_y_train,          \n    random_state=42\n)\n\n#Logistic Regression\ngrid_search_logistic.fit(DOS_x_train_subset, DOS_y_train_subset)\nDOS_logistic_bestestimator = grid_search_logistic.best_estimator_\nDOS_logistic_bestparams = grid_search_logistic.best_params_\nDOS_logistic_bestscore = grid_search_logistic.best_score_\nprint(\'DOS\')\nprint(\'Logistic Regression\')\nprint("Best parameters:", DOS_logistic_bestparams)\nprint("Best cross-val score: {:.2f}%".format(DOS_logistic_bestscore * 100))\nprint(\'\n\')\n\n#KNN\ngrid_search_knn.fit(DOS_x_train_subset, DOS_y_train_subset)\nDOS_knn_bestestimator = grid_search_knn.best_estimator_\nDOS_knn_bestparams = grid_search_knn.best_params_\nDOS_knn_bestscore = grid_search_knn.best_score_\nprint(\'KNN\')\nprint("Best parameters:", DOS_knn_bestparams)\nprint("Best score: {:.2f}%".format(DOS

#### WebAttack Analysis

In [29]:
#Hyperparameter training subset split
"""
WebAttack_x_train_subset, _, WebAttack_y_train_subset, _ = train_test_split(
    WebAttackMon_x_train, WebAttackMon_y_train,
    train_size=0.05,            
    stratify=WebAttackMon_y_train,          
    random_state=42
)

#Logistic Regression
grid_search_logistic.fit(WebAttack_x_train_subset, WebAttack_y_train_subset)
WebAttack_logistic_bestestimator = grid_search_logistic.best_estimator_
WebAttack_logistic_bestparams = grid_search_logistic.best_params_
WebAttack_logistic_bestscore = grid_search_logistic.best_score_
print('WebAttack')
print('Logistic Regression')
print("Best parameters:", WebAttack_logistic_bestparams)
print("Best cross-val score: {:.2f}%".format(WebAttack_logistic_bestscore * 100))
print('\n')

#KNN
grid_search_knn.fit(WebAttack_x_train_subset, WebAttack_y_train_subset)
WebAttack_knn_bestestimator = grid_search_knn.best_estimator_
WebAttack_knn_bestparams = grid_search_knn.best_params_
WebAttack_knn_bestscore = grid_search_knn.best_score_
print('KNN')
print("Best parameters:", WebAttack_knn_bestparams)
print("Best score: {:.2f}%".format(WebAttack_knn_bestscore * 100))
print('\n')

#Random Forest Classifier
grid_search_rf.fit(WebAttack_x_train_subset, WebAttack_y_train_subset)
WebAttack_rf_bestestimator = grid_search_rf.best_estimator_
WebAttack_rf_bestparams = grid_search_rf.best_params_
WebAttack_rf_bestscore = grid_search_rf.best_score_
print('Random Forest')
print("Best parameters:", WebAttack_rf_bestparams)
print("Best score : {:.2f}%".format(WebAttack_rf_bestscore * 100))
print('\n')

#MLP Classifier
grid_search_mlp.fit(WebAttack_x_train_subset, WebAttack_y_train_subset)
WebAttack_mlp_bestestimator = grid_search_mlp.best_estimator_
WebAttack_mlp_bestparams = grid_search_mlp.best_params_
WebAttack_mlp_bestscore = grid_search_mlp.best_score_
print('Multi-Layer Perceptron')
print("Best parameters:", WebAttack_mlp_bestparams)
print("Best score: {:.2f}%".format(WebAttack_mlp_bestscore * 100))
"""

'\nWebAttack_x_train_subset, _, WebAttack_y_train_subset, _ = train_test_split(\n    WebAttackMon_x_train, WebAttackMon_y_train,\n    train_size=0.05,            \n    stratify=WebAttackMon_y_train,          \n    random_state=42\n)\n\n#Logistic Regression\ngrid_search_logistic.fit(WebAttack_x_train_subset, WebAttack_y_train_subset)\nWebAttack_logistic_bestestimator = grid_search_logistic.best_estimator_\nWebAttack_logistic_bestparams = grid_search_logistic.best_params_\nWebAttack_logistic_bestscore = grid_search_logistic.best_score_\nprint(\'WebAttack\')\nprint(\'Logistic Regression\')\nprint("Best parameters:", WebAttack_logistic_bestparams)\nprint("Best cross-val score: {:.2f}%".format(WebAttack_logistic_bestscore * 100))\nprint(\'\n\')\n\n#KNN\ngrid_search_knn.fit(WebAttack_x_train_subset, WebAttack_y_train_subset)\nWebAttack_knn_bestestimator = grid_search_knn.best_estimator_\nWebAttack_knn_bestparams = grid_search_knn.best_params_\nWebAttack_knn_bestscore = grid_search_knn.best_s

#### Infiltration Analysis

In [30]:
#Hyperparameter training subset split
"""
Infiltration_x_train_subset, _, Infiltration_y_train_subset, _ = train_test_split(
    InfiltrationMon_x_train, InfiltrationMon_y_train,
    train_size=0.1,            
    stratify=InfiltrationMon_y_train,          
    random_state=42
)

#Logistic Regression
grid_search_logistic.fit(Infiltration_x_train_subset, Infiltration_y_train_subset)
Infiltration_logistic_bestestimator = grid_search_logistic.best_estimator_
Infiltration_logistic_bestparams = grid_search_logistic.best_params_
Infiltration_logistic_bestscore = grid_search_logistic.best_score_
print('Infiltration')
print('Logistic Regression')
print("Best parameters:", Infiltration_logistic_bestparams)
print("Best cross-val score: {:.2f}%".format(Infiltration_logistic_bestscore * 100))
print('\n')

#KNN
grid_search_knn.fit(Infiltration_x_train_subset, Infiltration_y_train_subset)
Infiltration_knn_bestestimator = grid_search_knn.best_estimator_
Infiltration_knn_bestparams = grid_search_knn.best_params_
Infiltration_knn_bestscore = grid_search_knn.best_score_
print('KNN')
print("Best parameters:", Infiltration_knn_bestparams)
print("Best score: {:.2f}%".format(Infiltration_knn_bestscore * 100))
print('\n')

#Random Forest Classifier
grid_search_rf.fit(Infiltration_x_train_subset, Infiltration_y_train_subset)
Infiltration_rf_bestestimator = grid_search_rf.best_estimator_
Infiltration_rf_bestparams = grid_search_rf.best_params_
Infiltration_rf_bestscore = grid_search_rf.best_score_
print('Random Forest')
print("Best parameters:", Infiltration_rf_bestparams)
print("Best score : {:.2f}%".format(Infiltration_rf_bestscore * 100))
print('\n')

#MLP Classifier
grid_search_mlp.fit(Infiltration_x_train_subset, Infiltration_y_train_subset)
Infiltration_mlp_bestestimator = grid_search_mlp.best_estimator_
Infiltration_mlp_bestparams = grid_search_mlp.best_params_
Infiltration_mlp_bestscore = grid_search_mlp.best_score_
print('Multi-Layer Perceptron')
print("Best parameters:", Infiltration_mlp_bestparams)
print("Best score: {:.2f}%".format(Infiltration_mlp_bestscore * 100))
"""

'\nInfiltration_x_train_subset, _, Infiltration_y_train_subset, _ = train_test_split(\n    InfiltrationMon_x_train, InfiltrationMon_y_train,\n    train_size=0.1,            \n    stratify=InfiltrationMon_y_train,          \n    random_state=42\n)\n\n#Logistic Regression\ngrid_search_logistic.fit(Infiltration_x_train_subset, Infiltration_y_train_subset)\nInfiltration_logistic_bestestimator = grid_search_logistic.best_estimator_\nInfiltration_logistic_bestparams = grid_search_logistic.best_params_\nInfiltration_logistic_bestscore = grid_search_logistic.best_score_\nprint(\'Infiltration\')\nprint(\'Logistic Regression\')\nprint("Best parameters:", Infiltration_logistic_bestparams)\nprint("Best cross-val score: {:.2f}%".format(Infiltration_logistic_bestscore * 100))\nprint(\'\n\')\n\n#KNN\ngrid_search_knn.fit(Infiltration_x_train_subset, Infiltration_y_train_subset)\nInfiltration_knn_bestestimator = grid_search_knn.best_estimator_\nInfiltration_knn_bestparams = grid_search_knn.best_params_

#### Botnet Analysis

In [31]:
#Hyperparameter training subset split
"""
Botnet_x_train_subset, _, Botnet_y_train_subset, _ = train_test_split(
    BotnetMon_x_train, BotnetMon_y_train,
    train_size=0.05,            
    stratify=BotnetMon_y_train,          
    random_state=42
)

#Logistic Regression
grid_search_logistic.fit(Botnet_x_train_subset, Botnet_y_train_subset)
Botnet_logistic_bestestimator = grid_search_logistic.best_estimator_
Botnet_logistic_bestparams = grid_search_logistic.best_params_
Botnet_logistic_bestscore = grid_search_logistic.best_score_
print('Botnet')
print('Logistic Regression')
print("Best parameters:", Botnet_logistic_bestparams)
print("Best cross-val score: {:.2f}%".format(Botnet_logistic_bestscore * 100))
print('\n')

#KNN
grid_search_knn.fit(Botnet_x_train_subset, Botnet_y_train_subset)
Botnet_knn_bestestimator = grid_search_knn.best_estimator_
Botnet_knn_bestparams = grid_search_knn.best_params_
Botnet_knn_bestscore = grid_search_knn.best_score_
print('KNN')
print("Best parameters:", Botnet_knn_bestparams)
print("Best score: {:.2f}%".format(Botnet_knn_bestscore * 100))
print('\n')

#Random Forest Classifier
grid_search_rf.fit(Botnet_x_train_subset, Botnet_y_train_subset)
Botnet_rf_bestestimator = grid_search_rf.best_estimator_
Botnet_rf_bestparams = grid_search_rf.best_params_
Botnet_rf_bestscore = grid_search_rf.best_score_
print('Random Forest')
print("Best parameters:", Botnet_rf_bestparams)
print("Best score : {:.2f}%".format(Botnet_rf_bestscore * 100))
print('\n')

#MLP Classifier
grid_search_mlp.fit(Botnet_x_train_subset, Botnet_y_train_subset)
Botnet_mlp_bestestimator = grid_search_mlp.best_estimator_
Botnet_mlp_bestparams = grid_search_mlp.best_params_
Botnet_mlp_bestscore = grid_search_mlp.best_score_
print('Multi-Layer Perceptron')
print("Best parameters:", Botnet_mlp_bestparams)
print("Best score: {:.2f}%".format(Botnet_mlp_bestscore * 100))
"""

'\nBotnet_x_train_subset, _, Botnet_y_train_subset, _ = train_test_split(\n    BotnetMon_x_train, BotnetMon_y_train,\n    train_size=0.05,            \n    stratify=BotnetMon_y_train,          \n    random_state=42\n)\n\n#Logistic Regression\ngrid_search_logistic.fit(Botnet_x_train_subset, Botnet_y_train_subset)\nBotnet_logistic_bestestimator = grid_search_logistic.best_estimator_\nBotnet_logistic_bestparams = grid_search_logistic.best_params_\nBotnet_logistic_bestscore = grid_search_logistic.best_score_\nprint(\'Botnet\')\nprint(\'Logistic Regression\')\nprint("Best parameters:", Botnet_logistic_bestparams)\nprint("Best cross-val score: {:.2f}%".format(Botnet_logistic_bestscore * 100))\nprint(\'\n\')\n\n#KNN\ngrid_search_knn.fit(Botnet_x_train_subset, Botnet_y_train_subset)\nBotnet_knn_bestestimator = grid_search_knn.best_estimator_\nBotnet_knn_bestparams = grid_search_knn.best_params_\nBotnet_knn_bestscore = grid_search_knn.best_score_\nprint(\'KNN\')\nprint("Best parameters:", Botne

#### Portscan Analysis

In [32]:
#Hyperparameter training subset split
"""
Portscan_x_train_subset, _, Portscan_y_train_subset, _ = train_test_split(
    PortscanMon_x_train, PortscanMon_y_train,
    train_size=0.05,            
    stratify=PortscanMon_y_train,          
    random_state=42
)

#Logistic Regression
grid_search_logistic.fit(Botnet_x_train_subset, Botnet_y_train_subset)
Portscan_logistic_bestestimator = grid_search_logistic.best_estimator_
Portscan_logistic_bestparams = grid_search_logistic.best_params_
Portscan_logistic_bestscore = grid_search_logistic.best_score_
print('Portscan')
print('Logistic Regression')
print("Best parameters:", Portscan_logistic_bestparams)
print("Best cross-val score: {:.2f}%".format(Portscan_logistic_bestscore * 100))
print('\n')

#KNN
grid_search_knn.fit(Portscan_x_train_subset, Portscan_y_train_subset)
Portscan_knn_bestestimator = grid_search_knn.best_estimator_
Portscan_knn_bestparams = grid_search_knn.best_params_
Portscan_knn_bestscore = grid_search_knn.best_score_
print('KNN')
print("Best parameters:", Portscan_knn_bestparams)
print("Best score: {:.2f}%".format(Portscan_knn_bestscore * 100))
print('\n')

#Random Forest Classifier
grid_search_rf.fit(Botnet_x_train_subset, Botnet_y_train_subset)
Portscan_rf_bestestimator = grid_search_rf.best_estimator_
Portscan_rf_bestparams = grid_search_rf.best_params_
Portscan_rf_bestscore = grid_search_rf.best_score_
print('Random Forest')
print("Best parameters:", Portscan_rf_bestparams)
print("Best score : {:.2f}%".format(Portscan_rf_bestscore * 100))
print('\n')

#MLP Classifier
grid_search_mlp.fit(Portscan_x_train_subset, Portscan_y_train_subset)
Portscan_mlp_bestestimator = grid_search_mlp.best_estimator_
Portscan_mlp_bestparams = grid_search_mlp.best_params_
Portscan_mlp_bestscore = grid_search_mlp.best_score_
print('Multi-Layer Perceptron')
print("Best parameters:", Portscan_mlp_bestparams)
print("Best score: {:.2f}%".format(Portscan_mlp_bestscore * 100))
"""

'\nPortscan_x_train_subset, _, Portscan_y_train_subset, _ = train_test_split(\n    PortscanMon_x_train, PortscanMon_y_train,\n    train_size=0.05,            \n    stratify=PortscanMon_y_train,          \n    random_state=42\n)\n\n#Logistic Regression\ngrid_search_logistic.fit(Botnet_x_train_subset, Botnet_y_train_subset)\nPortscan_logistic_bestestimator = grid_search_logistic.best_estimator_\nPortscan_logistic_bestparams = grid_search_logistic.best_params_\nPortscan_logistic_bestscore = grid_search_logistic.best_score_\nprint(\'Portscan\')\nprint(\'Logistic Regression\')\nprint("Best parameters:", Portscan_logistic_bestparams)\nprint("Best cross-val score: {:.2f}%".format(Portscan_logistic_bestscore * 100))\nprint(\'\n\')\n\n#KNN\ngrid_search_knn.fit(Portscan_x_train_subset, Portscan_y_train_subset)\nPortscan_knn_bestestimator = grid_search_knn.best_estimator_\nPortscan_knn_bestparams = grid_search_knn.best_params_\nPortscan_knn_bestscore = grid_search_knn.best_score_\nprint(\'KNN\')\

#### DDOS Analysis

In [33]:
#Hyperparameter training subset split
"""
DDOS_x_train_subset, _, DDOS_y_train_subset, _ = train_test_split(
    DDOSMon_x_train, DDOSMon_y_train,
    train_size=0.05,            
    stratify=DDOSMon_y_train,          
    random_state=42
)

#Logistic Regression
grid_search_logistic.fit(DDOS_x_train_subset, DDOS_y_train_subset)
DDOS_logistic_bestestimator = grid_search_logistic.best_estimator_
DDOS_logistic_bestparams = grid_search_logistic.best_params_
DDOS_logistic_bestscore = grid_search_logistic.best_score_
print('DDOS')
print('Logistic Regression')
print("Best parameters:", DDOS_logistic_bestparams)
print("Best cross-val score: {:.2f}%".format(DDOS_logistic_bestscore * 100))
print('\n')

#KNN
grid_search_knn.fit(DDOS_x_train_subset, DDOS_y_train_subset)
DDOS_knn_bestestimator = grid_search_knn.best_estimator_
DDOS_knn_bestparams = grid_search_knn.best_params_
DDOS_knn_bestscore = grid_search_knn.best_score_
print('KNN')
print("Best parameters:", DDOS_knn_bestparams)
print("Best score: {:.2f}%".format(DDOS_knn_bestscore * 100))
print('\n')

#Random Forest Classifier
grid_search_rf.fit(DDOS_x_train_subset, DDOS_y_train_subset)
DDOS_rf_bestestimator = grid_search_rf.best_estimator_
DDOS_rf_bestparams = grid_search_rf.best_params_
DDOS_rf_bestscore = grid_search_rf.best_score_
print('Random Forest')
print("Best parameters:", DDOS_rf_bestparams)
print("Best score : {:.2f}%".format(DDOS_rf_bestscore * 100))
print('\n')

#MLP Classifier
grid_search_mlp.fit(DDOS_x_train_subset, DDOS_y_train_subset)
DDOS_mlp_bestestimator = grid_search_mlp.best_estimator_
DDOS_mlp_bestparams = grid_search_mlp.best_params_
DDOS_mlp_bestscore = grid_search_mlp.best_score_
print('Multi-Layer Perceptron')
print("Best parameters:", DDOS_mlp_bestparams)
print("Best score: {:.2f}%".format(DDOS_mlp_bestscore * 100))
"""

'\nDDOS_x_train_subset, _, DDOS_y_train_subset, _ = train_test_split(\n    DDOSMon_x_train, DDOSMon_y_train,\n    train_size=0.05,            \n    stratify=DDOSMon_y_train,          \n    random_state=42\n)\n\n#Logistic Regression\ngrid_search_logistic.fit(DDOS_x_train_subset, DDOS_y_train_subset)\nDDOS_logistic_bestestimator = grid_search_logistic.best_estimator_\nDDOS_logistic_bestparams = grid_search_logistic.best_params_\nDDOS_logistic_bestscore = grid_search_logistic.best_score_\nprint(\'DDOS\')\nprint(\'Logistic Regression\')\nprint("Best parameters:", DDOS_logistic_bestparams)\nprint("Best cross-val score: {:.2f}%".format(DDOS_logistic_bestscore * 100))\nprint(\'\n\')\n\n#KNN\ngrid_search_knn.fit(DDOS_x_train_subset, DDOS_y_train_subset)\nDDOS_knn_bestestimator = grid_search_knn.best_estimator_\nDDOS_knn_bestparams = grid_search_knn.best_params_\nDDOS_knn_bestscore = grid_search_knn.best_score_\nprint(\'KNN\')\nprint("Best parameters:", DDOS_knn_bestparams)\nprint("Best score: 

#### Best Model Performances

In [34]:
datasets = {
    "Brute Force": (BruteForceMon_x_train, BruteForceMon_x_test, BruteForceMon_y_train, BruteForceMon_y_test),
    "DoS": (DOSMon_x_train, DOSMon_x_test, DOSMon_y_train, DOSMon_y_test),
    "WebAttack": (WebAttackMon_x_train, WebAttackMon_x_test, WebAttackMon_y_train, WebAttackMon_y_test),
    "Infiltration": (InfiltrationMon_x_train, InfiltrationMon_x_test, InfiltrationMon_y_train, InfiltrationMon_y_test),
    "Botnet": (BotnetMon_x_train, BotnetMon_x_test, BotnetMon_y_train, BotnetMon_y_test),
    "Portscan": (PortscanMon_x_train, PortscanMon_x_test, PortscanMon_y_train, PortscanMon_y_test),
    "DDOS": (DDOSMon_x_train, DDOSMon_x_test, DDOSMon_y_train, DDOSMon_y_test)
}

# Best hyperparameters
best_params_logistic = {
    "Brute Force": {'C': 10, 'penalty': 'l1', 'solver': 'saga'},
    "DoS": {'C': 10, 'penalty': 'l1', 'solver': 'saga'},
    "WebAttack": {'C': 10, 'penalty': 'l1', 'solver': 'saga'},
    "Infiltration": {'C': 10, 'penalty': 'l1', 'solver': 'saga'},
    "Botnet": {'C': 10, 'penalty': 'l2', 'solver': 'saga'},
    "Portscan": {'C': 10, 'penalty': 'l1', 'solver': 'saga'},
    "DDOS": {'C': 10, 'penalty': 'l1', 'solver': 'saga'}
}

best_params_knn = {
    "Brute Force": {'algorithm': 'auto', 'leaf_size': 30, 'n_neighbors': 7, 'weights': 'distance'},
    "DoS": {'algorithm': 'auto', 'leaf_size': 30, 'n_neighbors': 9, 'weights': 'distance'},
    "WebAttack": {'algorithm': 'auto', 'leaf_size': 30, 'n_neighbors': 7, 'weights': 'distance'},
    "Infiltration": {'algorithm': 'auto', 'leaf_size': 30, 'n_neighbors': 3, 'weights': 'uniform'},
    "Botnet": {'algorithm': 'auto', 'leaf_size': 30, 'n_neighbors': 9, 'weights': 'distance'},
    "Portscan": {'algorithm': 'auto', 'leaf_size': 30, 'n_neighbors': 3, 'weights': 'distance'},
    "DDOS": {'algorithm': 'auto', 'leaf_size': 30, 'n_neighbors': 7, 'weights': 'distance'}
}

best_params_rf = {
    "Brute Force": {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200},
    "DoS": {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 500},
    "WebAttack": {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 500},
    "Infiltration": {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 500},
    "Botnet": {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200},
    "Portscan": {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200},
    "DDOS": {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}
}

best_params_mlp = {
    "Brute Force": {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (128, 64), 'learning_rate': 'adaptive', 'solver': 'adam'},
    "DoS": {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (128, 64), 'learning_rate': 'adaptive', 'solver': 'adam'},
    "WebAttack": {'activation': 'relu', 'alpha': 0.01, 'hidden_layer_sizes': (128,), 'learning_rate': 'adaptive', 'solver': 'adam'},
    "Infiltration": {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (128, 64), 'learning_rate': 'adaptive', 'solver': 'adam'},
    "Botnet": {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (128, 64), 'learning_rate': 'adaptive', 'solver': 'adam'},
    "Portscan": {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (128, 64), 'learning_rate': 'adaptive', 'solver': 'adam'},
    "DDOS": {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (128, 64), 'learning_rate': 'adaptive', 'solver': 'adam'}
}

# Helper function to generate table
def evaluate_model_table(model_name, model_class, best_params_dict):
    rows = []
    for attack_name, (X_train, X_test, y_train, y_test) in datasets.items():
        params = best_params_dict[attack_name]
        model = model_class(**params)

        # Special arguments
        if model_name == "Logistic Regression":
            model.max_iter = 2000
            model.n_jobs = -1
            model.class_weight = "balanced"
        if model_name == "Random Forest":
            model.n_jobs = -1
            model.class_weight = "balanced"
        if model_name == "MLP":
            model.max_iter = 2000

        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        if hasattr(model, "predict_proba"):
            probs = model.predict_proba(X_test)[:, 1]
        else:
            probs = preds

        acc = accuracy_score(y_test, preds)
        f1 = f1_score(y_test, preds, average="binary")
        try:
            roc_auc = roc_auc_score(y_test, probs)
            avg_prec = average_precision_score(y_test, probs)
        except:
            roc_auc = None
            avg_prec = None

        row = {
            "Attack Type": attack_name,
            **params,
            "Accuracy": round(acc,4),
            "F1 Score": round(f1,4),
            "ROC AUC": round(roc_auc,4) if roc_auc is not None else None,
            "Average Precision": round(avg_prec,4) if avg_prec is not None else None
        }
        rows.append(row)

    df = pd.DataFrame(rows)
    print(f"\n==== {model_name} Table ====\n")
    print(tabulate(df, headers='keys', tablefmt='fancy_grid', showindex=False))
    return df

# Generate tables for all models
logistic_table = evaluate_model_table("Logistic Regression", LogisticRegression, best_params_logistic)
knn_table = evaluate_model_table("KNN", KNeighborsClassifier, best_params_knn)
rf_table = evaluate_model_table("Random Forest", RandomForestClassifier, best_params_rf)
mlp_table = evaluate_model_table("MLP", MLPClassifier, best_params_mlp)

/opt/homebrew/Caskroom/miniforge/base/envs/cs135_env/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/cs135_env/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/cs135_env/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/cs135_env/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/cs135_env/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarni


==== Logistic Regression Table ====

╒═══════════════╤═════╤═══════════╤══════════╤════════════╤════════════╤═══════════╤═════════════════════╕
│ Attack Type   │   C │ penalty   │ solver   │   Accuracy │   F1 Score │   ROC AUC │   Average Precision │
╞═══════════════╪═════╪═══════════╪══════════╪════════════╪════════════╪═══════════╪═════════════════════╡
│ Brute Force   │  10 │ l1        │ saga     │     0.8765 │     0.1866 │    0.9954 │              0.8432 │
├───────────────┼─────┼───────────┼──────────┼────────────┼────────────┼───────────┼─────────────────────┤
│ DoS           │  10 │ l1        │ saga     │     0.9259 │     0.8463 │    0.9891 │              0.9645 │
├───────────────┼─────┼───────────┼──────────┼────────────┼────────────┼───────────┼─────────────────────┤
│ WebAttack     │  10 │ l1        │ saga     │     0.9323 │     0.0823 │    0.9877 │              0.1486 │
├───────────────┼─────┼───────────┼──────────┼────────────┼────────────┼───────────┼─────────────────────┤

#### Hyperparameter visualization